<a href="https://colab.research.google.com/github/Davre-ML/AppTareasKivy/blob/main/info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# App de tareas con Kivy

### 1.-Constantes de color

```python
BG_COLOR = (1, 1, 1, 1) # Blanco (R, G, B, A)
FG_COLOR = (0, 0, 0, 1) # Negro
```

Estas líneas definen las tuplas de colores para el fondo (`BG_COLOR`) y el primer plano/texto (`FG_COLOR`). Usar constantes facilita los colores en un solo lugar y mejora la legibilidad del código. Los valores son en formato RGBA, donde cada componente va de 0 a 1

## 2.-Clase `TaskWidget`

Esta clase representa un solo elemento de tarea en la lista. Cada vez que añades una tarea, se crea una instancia de `TaskWidget` .

### `__init__(self, text, completed=False, **kwargs)` :

**Propósito:** Constructor de la clase **`TaskWidget`** . Se ejecuta cuando se crea una nueva instancia de una tarea.

- **Lógica:**
    - Llama al constructor de la clase padre (`BoxLayout`) para inicializar el layout
    - `self, task_text = text` : Almacena el texto de la tarea. `StringProperty` hace que Kivy pueda observar cambios en esta propiedad.
    - `self.is_completed = completed` :Almacena es estado de completado de la tarea (True/False). `BooleanProperty` también es observable.
    - Configura el `BoxLayout` para ser horizontal, con una altura fija (`dp(55)`) y un espacio/padding.
    - **Dibujo de fondo:** `with self.canvas.before:` se usa para dibujar un rectangulo de color detrás del `BoxLayout`no tiene una propiedad `background_color` directa. `self.bind(pos=..., size=...)` asegura que el rectángulo se redibuje si el widget cambia de posición o tamaño.
    - `complete_button` : Crea un `button` para marcar la tarea como completada.
        - Su texto inicial es '◻️’ (cuadrado vacío)
        - Se le da un tamaño fijo, se elimina el fondo normal de Kivy (`background_normal=''`) y se le asigna un color de fondo negro y texto blanco.
        - `bind(on_release=self.toggle_completion)`: Asocia el método `toggle_completion` para que se ejecute cunado el botón sea liberado
    - `task_label` : Crea un label para mostrar el texto de la tarea.
        - `hagin='left'` ,`valign='middle'` : Alinea el texto a la izquierda y al centro verticalmente
        - `text_size` : Importante para que el texto se ajuste al ancho disponible y no se salga del widget.
        - `color=FG_COLOR` : Establece el color del texto a negro
        - `mark=True` : Crucial para permitir el uso de etiquetas de formato `[s][/s]` para tachar el texto
        - `bind(wigth=self._update_label_text_size)` : Asegura que el `text_size` de la etiqueta se ajuste si el ancho del `TaskWidget`cambia
    - `delete_button` :Crea un `button` para eliminar la tarea
        - Texto ‘X’, tamaño fijo, fondo rojo y texto blanco
        - `bind(on_release=self.deleate_task)`: Asocia el método `delete_task`
    - `self._update_ui_for_completion()` : Llama a este método al inicio para asegurar que la UI refleje el estado inicial de `is_completed` .

### `_update_rect(self, instance, value)` :

**Propósito:** Actualiza la posición y el tamaño del rectángulo de fondo del `TaskWidget` para que coincida con el widget.

**Lógica:** Es un método auxiliar llamado por los eventos `pos` y`size` del widget, asegurando que el fondo se dibuje correctamente.

### `_update_label_text_size(self, instance, value)` :

**Propósito:** Ajusta el **`text_size`** de la **`task_label`** cada vez que el ancho de la `TaskWidget` cambia

**Lógica:** Esto es vital para que el texto de la tarea se envuelva correctamente y no se desborde si la ventana de la aplicación se redimenciona

### `toggle_completion(self, instnce)` :

**Propósito:** Cambia el estado de completado de la tarea y actualiza su apariencia

**Lógica:**

- `self.is_completed = not self.is_completed` : Invierte el valor booleano de `is_completed`
- `self._update_ui_for_completion()` : Llama a la función para redibujar el widget con el nuevo estado
- `App.get_running_app().save_tasks()` : Importante para el guardado; llama al método `save_tasks` de la aplicación principal para guardar el estado actualizado de todas las tareas.

### `_update_ui_for_completion(self)` :

**Propósito:** Modifica la apariencia del **`TaskWidget`** según si la tarea está completada o pendiente

**Lógica:**

- Si `is_completed` es`True` :
    - El botón de completado cambia a '✅'.
    - El color del texto de la etiqueta se vuelve gris (`(0.5, 0.5, 0.5, 1)`).
    - El texto de la etiqueta se envuelve con `[s][/s]` para aplicarle un tachado.
    - El color de fondo del `TaskWidget` cambia a un gris muy claro (`(0.95, 0.95, 0.95, 1)`).
- Si `is_completed` es `False`:
    - El botón de completado vuelve a '◻️'.
    - El color del texto de la etiqueta vuelve a negro (`FG_COLOR`).
    - El texto de la etiqueta vuelve a su forma original (sin tachado).
    - El color de fondo del `TaskWidget` vuelve a blanco (`BG_COLOR`).
- `self.rect_color.rgba = self.background_color`: Actualiza el color del rectángulo de fondo.

### `delete_task(sef, instance)` :

**Propósito:** Inicia el proceso para eliminar una tarea

**Lógica:** Llama al método `confirm_delete_task` de la aplicación principal, pasándose a sí mismo `self` como el widget a eliminar. Esto permite que la lógica de confirmación y eliminación real resida en la aplicación principal.

<aside>
🔥

Para que `delete_task` funcione, el `TaskWidget` necesita una forma de comunicarse con su padre (`TasAPP`) para que esto lo elimine de la lista. En el ejemplo, lo hice a través de`App.get_running_app()` , que es un poco un “hack” para ejemplos rápidos. Una forma más robusta sería pasar una referencia a la función de eliminación como callback cuando creas el `TaskWidget`

</aside>

## 3.-Clase `TaskApp`

### `build(self)` :

**Propósito:** Es el método central de Kivy donde se construye la interfaz de usuario de la aplicación. Se llama una vez cuando la aplicación se inicia.

**Lógica:**

`window.clearcolor=BG_COLOR` : Establece el color de fondo de toda la ventana de Kivy en blanco

`main_layout` : Un `boxlayout` vertical que contendrá todos los elementos principales de la UI (título, entrada de tarea, lista de tareas). Se le da padding y espaciado. También se le dibuja un fondo blanco

`title_label` : Una `label` para el título “Mis Tareas”, con una fuente grande y negrita, color negro

`add_task_layout:` Un `BoxLayout` horizontal para el campo de entrada de texto y el botón añadir.

- `self.new_task_input` :Un `Textinput` donde el usuario escribe nuevas tareas. Se le configura con una fuente grande, fondo blanco, texto negro y cursor negro.
- `add.button` : Un botón para añadir la tarea. Se le configura con un fondo negro y texto blanco. `bind(on_release=self.add_task)` asocia el método `add_task`

`self.task_list_container` : Otro `boxlayout` vertical. Este será el contenedor de todos los `widget` individuales:

- `bind(minimum_height=...)` : Es crucial para que el `Scrollview` funcione correctamente. Le dice al `Scrollview` que el `task_list_container` debe expandirse verticalmente para contener todos sus hijos.

`scroll_view` : Un `Scrollview` que permite que el `task_list_container` sea desplazable si su contenido excede el tamaño de la pantalla

`self.load_tasks()` : Llama a este método para cargar las tareas de ejemplo al inicio de la aplicación

Retorna `main_layout` , que es la raíz de la interfaz de usuario

### `_biuld_main_rect(self, instance, value)` :

**Propósito:** Actualiza la posición y el tamaño del rectángulo de fondo del `main_layout` para que coincida con el layout.

**Lógica:** Similar a `_update_rect` en `TaskWidget`.

### **`add_task(self, instance)`:**

**Propósito:** Maneja la lógica para añadir una nueva tarea.

**Lógica:**

- Obtiene el texto del `new_task_input` y le quita espacios en blanco al inicio/final (`strip()`).
- Si el texto no está vacío:
    - Crea una nueva instancia de `TaskWidget` con el texto.
    - Añade este `task_widget` al `self.task_list_container`.
    - Limpia el `new_task_input`.
    - Llama a `self.save_tasks()` para guardar el estado actualizado.
- Si el texto está vacío, no hace nada (en una app más compleja, aquí podrías mostrar un mensaje al usuario).

### **`confirm_delete_task(self, task_widget_to_delete)`:**

**Propósito:** Maneja la confirmación y eliminación de una tarea.

**Lógica:** En este ejemplo simplificado, llama directamente a `self.remove_task_widget`. En una aplicación real, Kivy no tiene un `messagebox` nativo como Tkinter, por lo que aquí se implementaría un `Popup` personalizado para pedir confirmación al usuario antes de eliminar la tarea.

### **`remove_task_widget(self, task_widget)`:**

**Propósito:** Elimina un `TaskWidget` de la lista de tareas.

**Lógica:**

- `self.task_list_container.remove_widget(task_widget)`: Elimina visualmente el widget del contenedor.
- `self.save_tasks()`: Llama a `save_tasks()` para actualizar el estado guardado.

### **`load_tasks(self)`:**

**Propósito:** Carga las tareas existentes al iniciar la aplicación.

**Lógica:**

- Contiene una lista `example_tasks` con datos de tareas predefinidas (texto y estado de completado).
- Itera sobre esta lista y para cada tarea:
    - Crea un `TaskWidget` con el texto y el estado de completado.
    - Añade el `TaskWidget` al `task_list_container`.

### **`save_tasks(self)`:**

**Propósito:** Guarda el estado actual de todas las tareas.

**Lógica para el guardado de tareas:**

- Crea una lista vacía `tasks_to_save`.
- Itera sobre `self.task_list_container.children`. **Importante:** Kivy almacena los widgets hijos en orden inverso al que fueron añadidos.
- Para cada `widget` en los hijos:
    - Verifica si el `widget` es una instancia de `TaskWidget` (para asegurarse de que solo se guarden las tareas).
    - Si lo es, crea un diccionario con el `text` y el estado `completed` de esa tarea.
    - Añade este diccionario a `tasks_to_save`.
- `tasks_to_save.reverse()`: Invierte la lista para que el orden de las tareas guardadas coincida con el orden en que fueron añadidas por el usuario.
- `print("Tareas guardadas (simulado):", tasks_to_save)`: **En este ejemplo, solo imprime la lista de tareas en la consola.**
    - **Para una persistencia real:** Aquí es donde integrarías la lógica para escribir esta lista `tasks_to_save` en un archivo (por ejemplo, usando el módulo `json` de Python para guardarlo como un archivo JSON) o en una base de datos (como SQLite).
    - La idea es que cada vez que se añade, se marca como completada o se elimina una tarea, esta función se llama para que el estado de la aplicación se actualice en el almacenamiento permanente.

### **`on_stop(self)`:**

**Propósito:** Un método de ciclo de vida de la aplicación Kivy que se llama justo antes de que la aplicación se cierre.

**Lógica:** Llama a `self.save_tasks()` para asegurar que el estado final de las tareas se guarde cuando el usuario cierra la aplicación.

Para finalizar, les agradezco que hayan leído todo esto.